In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/stop-words-in-28-languages/vietnamese.txt
/kaggle/input/stop-words-in-28-languages/bulgarian.txt
/kaggle/input/stop-words-in-28-languages/ukrainian.txt
/kaggle/input/stop-words-in-28-languages/swedish.txt
/kaggle/input/stop-words-in-28-languages/danish.txt
/kaggle/input/stop-words-in-28-languages/dutch.txt
/kaggle/input/stop-words-in-28-languages/russian.txt
/kaggle/input/stop-words-in-28-languages/slovak.txt
/kaggle/input/stop-words-in-28-languages/hungarian.txt
/kaggle/input/stop-words-in-28-languages/czech.txt
/kaggle/input/stop-words-in-28-languages/turkish.txt
/kaggle/input/stop-words-in-28-languages/malaysian.txt
/kaggle/input/stop-words-in-28-languages/italian.txt
/kaggle/input/stop-words-in-28-languages/catalan.txt
/kaggle/input/stop-words-in-28-languages/arabic.txt
/kaggle/input/stop-words-in-28-languages/norwegian.txt
/kaggle/input/stop-words-in-28-languages/hebrew.txt
/kaggle/input/stop-words-in-28-languages/gujarati.txt
/kaggle/input/stop-words-in-28-languages

#Codes by https://www.kaggle.com/anilreddy8989/word-guess-with-rnn

#Package Imports

In [2]:
from __future__ import absolute_import, division, print_function
import tensorflow.compat.v1 as tf
#import tensorflow as tf
tf.enable_eager_execution()


import time

In [3]:
# Adding file path
file_path = '../input/stop-words-in-28-languages/english.txt'

# Reading the file data, and then decoding to py2 compat.
text_data = open(file_path, 'rb').read().decode(encoding='utf-8')

#Data Processing

In [4]:
# The unique characters in the file
vocab_unique = sorted(set(text_data))

# Unique characters to Indices Mapping
char2ind = {u:i for i, u in enumerate(vocab_unique)}
ind2char = np.array(vocab_unique)

text_to_int = np.array([char2ind[c] for c in text_data])

In [5]:
# The maximum length sentence we want for a single input in characters
seq_len = 50
text_for_epoch = len(text_data)//seq_len

# Creating training targets
target_dataset = tf.data.Dataset.from_tensor_slices(text_to_int)

for i in target_dataset.take(5):
    print(ind2char[i.numpy()])

'
l
l


'


In [6]:
sequence = target_dataset.batch(seq_len+1, drop_remainder=True)

def splitting_target_input(chunk):
    text_input = chunk[:-1]
    text_target = chunk[1:]
    return text_input, text_target

dataset = sequence.map(splitting_target_input)

In [7]:
for input_example, target_example in  dataset.take(1):
    print ('Input data: ', repr(''.join(ind2char[input_example.numpy()])))
    print ('Target data:', repr(''.join(ind2char[target_example.numpy()])))

Input data:  "'ll\n'tis\n'twas\n've\n10\n39\na\na's\nable\nableabout\nabou"
Target data: "ll\n'tis\n'twas\n've\n10\n39\na\na's\nable\nableabout\nabout"


In [8]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(ind2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(ind2char[target_idx])))

Step    0
  input: 1 ("'")
  expected output: 19 ('l')
Step    1
  input: 19 ('l')
  expected output: 19 ('l')
Step    2
  input: 19 ('l')
  expected output: 0 ('\n')
Step    3
  input: 0 ('\n')
  expected output: 1 ("'")
Step    4
  input: 1 ("'")
  expected output: 27 ('t')


In [9]:
# Batch size 
BATCH_SIZE = 64
steps_per_epoch = text_for_epoch//BATCH_SIZE
BUFFER_SIZE = 10000
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

In [10]:
vocab_size = len(vocab_unique)
embedding_dim = 256
rnn_units = 1024
rnn = tf.keras.layers.CuDNNGRU 

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, 
                              batch_input_shape=[BATCH_SIZE, None]),
    rnn(rnn_units,
        return_sequences=True, 
        recurrent_initializer='glorot_uniform',
        stateful=True),
    tf.keras.layers.Dense(vocab_size)
  ])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           8960      
_________________________________________________________________
cu_dnngru (CuDNNGRU)         (64, None, 1024)          3938304   
_________________________________________________________________
dense (Dense)                (64, None, 35)            35875     
Total params: 3,983,139
Trainable params: 3,983,139
Non-trainable params: 0
_________________________________________________________________


#From now Only Errors. I saved the script for another Dataset. 

In [11]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)

NotFoundError: Could not find device for node: {{node CudnnRNNV2}} = CudnnRNNV2[T=DT_FLOAT, direction="unidirectional", dropout=0, input_mode="linear_input", is_training=true, rnn_mode="gru", seed=0, seed2=0]
All kernels registered for op CudnnRNNV2:
  <no registered kernels>
 [Op:CudnnRNNV2]

In [12]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

NameError: name 'example_batch_predictions' is not defined

In [13]:
# Train the Model

def loss_func(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss_func(target_example_batch, example_batch_predictions)

model.compile(
    optimizer = tf.train.AdamOptimizer(),
    loss = loss_func)

NameError: name 'example_batch_predictions' is not defined

In [14]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [15]:
EPOCHS=10

history = model.fit(dataset.repeat(), epochs=EPOCHS, steps_per_epoch=steps_per_epoch, callbacks=[checkpoint_callback])

RuntimeError: You must compile your model before training/testing. Use `model.compile(optimizer, loss)`.

In [16]:
tf.train.latest_checkpoint(checkpoint_dir)


model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, 
                              batch_input_shape=[1, None]),
    rnn(rnn_units,
        return_sequences=True, 
        recurrent_initializer='glorot_uniform',
        stateful=True),
    tf.keras.layers.Dense(vocab_size)
  ])



model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))


model.summary()

AttributeError: 'NoneType' object has no attribute 'endswith'

In [17]:
# Number of characters to generate
max_words = 350

# Starting String
start_string=u"ANTONIO: "



input_eval = [char2ind[s] for s in start_string]
input_eval = tf.expand_dims(input_eval, 0)


gen_text = []

temperature = 1.0


model.reset_states()
for i in range(max_words):
    predictions = model(input_eval)

    predictions = tf.squeeze(predictions, 0)


    predictions = predictions / temperature
    predicted_id = tf.multinomial(predictions, num_samples=1)[-1,0].numpy()

  
    input_eval = tf.expand_dims([predicted_id], 0)

    gen_text.append(ind2char[predicted_id])
    
print(start_string + ''.join(gen_text))

KeyError: 'A'